In [3]:
!pip install bs4

In [6]:
!pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pandas

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def getdata(URL):
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.4; x64; en-US) AppleWebKit/601.48 (KHTML, like Gecko) Chrome/49.0.2767.228 Safari/602.0 Edge/13.57484', 'Accept-Language': 'en-Us, en;q=0.5'}
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    return soup

def get_title(soup):
    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_price(soup):
    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

def get_next_page(soup):
    page = soup.find('span', attrs={'class':'s-pagination-strip'})
    
    if not page.find('span', attrs={'class':'s-pagination-item s-pagination-next s-pagination-disabled '}):
        URL = 'https://www.amazon.in/'+ str(page.find('span', attrs={'class':'s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'}).find('a')['href'])
        return URL
    
    else:
        return
    
URL = get_next_page(soup)
    
while True:
    soup = getdata(URL)
    URL1 = getnextpage(soup)
    if not URL1:
        break
    get_next_page(URL1)

if __name__ == '__main__':
    starting_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    data = []

    while starting_url:
        soup = getdata(starting_url)
        links = soup.find_all("a", class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')

        for link in links:
            new_url = "https://www.amazon.in" + link.get('href')
            new_soup = getdata(new_url)
            title = get_title(new_soup)
            price = get_price(new_soup)
            rating = get_rating(new_soup)
            review_count = get_review_count(new_soup)
            data.append({'title': title, 'price': price, 'rating': rating, 'reviews': review_count})

        starting_url = get_next_page(soup)

    amazon_df = pd.DataFrame(data)
    amazon_df.dropna(subset=['title'], inplace=True)
    amazon_df.to_csv("amazon_bags_data.csv", header=True, index=False)
amazon_df

NameError: name 'soup' is not defined